# ISS Tracker
Just a test project to get a feel for clientside callbacks.
See ```@app.callback()```

In [1]:
from tqdm import tqdm
import branca.colormap as cm
import requests

import dash
from dash import Dash, html, dcc
import dash_leaflet as dl
from dash.dependencies import Input, Output, State

In [2]:
# make a json call to http://api.open-notify.org/iss-now.json
# the response will be a json object
# get latitude and longitude from iss_position in the dict

def get_iss_position():
    """
    Get the current position of the ISS from the Open Notify API
    """
    response = requests.get('http://api.open-notify.org/iss-now.json')
    data = response.json()
    position = data['iss_position']
    # parse the strings
    position['latitude'] = float(position['latitude'])
    position['longitude'] = float(position['longitude'])
    return position

print(get_iss_position())

{'longitude': 43.1796, 'latitude': -32.5412}


In [3]:
# david is a god for making this work
def create_awesome_marker(position=(0.0,0.0), popup=None, icon='satellite', color='darkblue') -> dl.DivMarker:
    """
    Create an awesome marker with a Font Awesome icon
    - feature: Feature from the database
    - style: Style from the database
    - popup: Popup html content as string
    - icon: Font Awesome icon name from https://fontawesome.com/icons
    - color: marker color as string. Possible values: ```{red, darkred, lightred, orange, beige, green, darkgreen,
    lightgreen, blue, darkblue, lightblue, purple, darkpurple, pink, cadetblue, white, gray, lightgray, black}```
    """

    children = []

    if popup is not None:
        children.append(dl.Popup(content=popup))

    awesome_marker = dl.DivMarker(
        position=position,
        children=children,
        iconOptions=dict(
            html=f'<i class="awesome-marker awesome-marker-icon-{color} leaflet-zoom-animated leaflet-interactive"></i>'
            f'<i class="fa fa-{icon} icon-white" aria-hidden="true" style="position: relative; top: 33% !important; left: 37% !important; transform: translate(-50%, -50%) scale(1.2);"></i>',
            className='custom-div-icon',
            iconSize=[20, 20],
            iconAnchor=[10, 30],
            tooltipAnchor=[10, -20],
            popupAnchor=[-3, -31]
        )
    )

    return awesome_marker

In [4]:
# get the current position of the ISS and create a marker
def get_iss_marker():
    """
    Get the current position of the ISS and create a marker
    """

    position = get_iss_position()

    popup = "<b>ISS</b><br>" \
            f"Latitude: {position['latitude']:.2f}<br>" \
            f"Longitude: {position['longitude']:.2f}"
    
    return create_awesome_marker(position=(position['latitude'], position['longitude']), popup=popup, icon='satellite', color='darkblue')

In [5]:
app = Dash(
    __name__,
    external_stylesheets=[
        'https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css',
        'http://code.ionicframework.com/ionicons/1.5.2/css/ionicons.min.css',
        'https://raw.githubusercontent.com/lennardv2/Leaflet.awesome-markers/2.0/develop/dist/leaflet.awesome-markers.css',
        'https://getbootstrap.com/1.0.0/assets/css/bootstrap-1.0.0.min.css',
    ],
    external_scripts=[
        'http://cdn.leafletjs.com/leaflet-0.6.4/leaflet.js',
        'https://kit.fontawesome.com/5ae05e6c33.js'
    ]
)

# get the initial position for the iss
position = get_iss_position()
(latitude, longitude) = (position['latitude'], position['longitude'])

@app.callback(
    Output('map', 'children'),
    [Input('interval-component', 'n_intervals')]
)
def update_map(n):
    iss_marker = get_iss_marker()
    position = get_iss_position()
    new_center = (position['latitude'], position['longitude'])

    map_children = [
        dl.TileLayer(
            url='https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',
        	attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>',
            subdomains='abcd',
            maxZoom=20
        ),
        iss_marker
    ]

    return [
        dl.Map(
            map_children,
            zoom=6,
            center=new_center,
            id='map',
            style={'width': '100vw', 'height': '100vh'},
            # Add properties to make the map static
            zoomControl=False,
            doubleClickZoom=False,
            closePopupOnClick=False,
            dragging=False,
            zoomSnap=False,
            zoomDelta=False,
            trackResize=False,
            touchZoom=False,
            scrollWheelZoom=False
        )
    ]

app.layout = html.Div([
    dl.Map(
        [            
            dl.TileLayer(
                    url='https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',
                	attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>',
                    subdomains='abcd',
                    maxZoom=20
                ),
            get_iss_marker()
        ],
        zoom=7,
        center=(latitude, longitude),
        id='map',
        style={'width': '100vw', 'height': '100vh'}
        ),
    dcc.Interval(
        id='interval-component',
        interval=5000, # in milliseconds
        n_intervals=0
    ),
    dcc.Store(id='map-center-store', data=(latitude, longitude))
    ],
)

if __name__ == '__main__':
    app.run()